In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils

In [3]:
import numpy as np
import pandas as pd
import random
from utils import pairs2dict, run_experiment__timeaware, AUC

In [4]:
df_item2artist = pd.read_csv('/mnt/workspace/Behance/Behance_Item_to_Owners', sep=" ",
                            header=None, names=['item', 'artist'])

In [5]:
item2artists = pairs2dict(df_item2artist.values)

In [6]:
def process_like(user, item, t):
    try:
        artist_freqs = user2artist_freqs[user]
    except KeyError:
        artist_freqs = user2artist_freqs[user] = dict()
    for a in item2artists[item]:
        artist_freqs[a] = artist_freqs.get(a, 0) + 1
        global_artist_freqs[a] = global_artist_freqs.get(a, 0) + 1

In [7]:
def fa_score(artist_freqs, item):    
    return (sum(artist_freqs.get(a,0) for a in item2artists[item]),\
            sum(global_artist_freqs.get(a,0) for a in item2artists[item]))
            

In [8]:
def compute_AUC(u, pos_items, neg_items, t):
    
    # rank test items
    pos_items_set = set(pos_items)
    all_items = pos_items + neg_items
    random.shuffle(all_items)    
    artist_freqs = user2artist_freqs[u]
    all_items.sort(key=lambda i: fa_score(artist_freqs, i), reverse=True)
    
    # compute AUC
    relevant_positions = [pos for pos, item in enumerate(all_items) if item in pos_items_set]
    return AUC(relevant_positions, len(all_items))

In [10]:
user2artist_freqs = dict()
global_artist_freqs = dict()
run_experiment__timeaware(process_like, compute_AUC, '/mnt/workspace/Behance/ExperimentResults/', 'FA(p+np)')

len(events) =  999997
experiment successfully finished: results saved to /mnt/workspace/Behance/ExperimentResults/FA(p+np)_aucs.npy
	 elapsed_seconds = 14.20, mean_AUC = 0.85261
